
## Create Table in Database

In [4]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text
import warnings

warnings.filterwarnings("ignore")

In [8]:


user='martin_svitek'
password='T67wni047Ugy2uUx'

In [9]:
# Conection
host = 'localhost'
port = '5433'
database = 'layereddb'
schema='berlin_source_data'

#connection to db after i opened tunnel
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

In [10]:
#create table with constraints and references first
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {schema}.test_table_{user} (
    id INTEGER PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    score INTEGER
);
"""

# Execute the query to create my empty table
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    conn.commit()  # commit the transaction

In [ ]:
#  Send the DataFrame to the database using .to_sql()
df.to_sql(
    f'veterinary_clinics_{user_name}',        # table name
    engine,
    schema=schema,
    if_exists='append', # append table if it exists
    index=False
)

print("DataFrame sent to PostgreSQL using .to_sql() with psycopg2!")

In [11]:
##let's query test data!
query = f"""
SELECT * from berlin_source_data.districts
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df


,district_id,district,geometry
0,11012012,Reinickendorf,0106000020E61000000100000001030000000100000084...
1,11004004,Charlottenburg-Wilmersdorf,0106000020E6100000010000000103000000010000000D...
2,11009009,Treptow-Köpenick,0106000020E610000001000000010300000001000000E9...
3,11003003,Pankow,0106000020E61000000400000001030000000100000012...
4,11008008,Neukölln,0106000020E610000001000000010300000001000000BF...
5,11011011,Lichtenberg,0106000020E61000000100000001030000000100000038...
6,11010010,Marzahn-Hellersdorf,0106000020E6100000010000000103000000010000005A...
7,11005005,Spandau,0106000020E6100000010000000103000000010000001D...
8,11006006,Steglitz-Zehlendorf,0106000020E6100000010000000103000000010000008A...
9,11001001,Mitte,0106000020E610000001000000010300000001000000F3...


In [12]:
# --- CONFIG ---
CSV_PATH   = "/Users/martinsvitek/layered-populate-data-pool-da/layered-populate-data-pool-da/vet_clinics/sources/vets_with_districts_neighborhoods.csv"   # <-- change this
USER_NAME  = "martin_svitek"                                  # <-- change this
SCHEMA     = "berlin_source_data"                       # or e.g. "berlin_source_data"
TABLE_NAME = f"veterinary_clinics_{USER_NAME}"

# --- LOAD & INSERT IN CHUNKS ---
chunksize = 50_000  # tune as needed
total_rows = 0
for i, chunk in enumerate(pd.read_csv(CSV_PATH, chunksize=chunksize)):

    chunk.to_sql(
        TABLE_NAME,
        engine,
        schema=SCHEMA,
        if_exists="append",
        index=False,
        method="multi",     # batches multi-row INSERTs
    )
    total_rows += len(chunk)
    print(f"Inserted chunk {i+1}: {len(chunk):,} rows (total {total_rows:,})")

print(f"✅ Done. Inserted ~{total_rows:,} rows into {SCHEMA}.{TABLE_NAME}")

# --- VERIFY ---
check = pd.read_sql(
    f"SELECT COUNT(*) AS rows FROM {SCHEMA}.{TABLE_NAME};", engine
)
print(check)


Inserted chunk 1: 178 rows (total 178)
✅ Done. Inserted ~178 rows into berlin_source_data.veterinary_clinics_martin_svitek
   rows
0   178


In [ ]:

query = """
SELECT *
FROM berlin_source_data.veterinary_clinics_martin_svitek
LIMIT 10
"""

df = pd.read_sql(text(query), engine)  
df  # last line in the cell -> Jupyter will render the table


,clinic_id,clinic_name,amenity,street,house_number,postcode,city,phone_number,website,email,...,emergency,latitude,longitude,district_id,neighbourhood_id,full_address,district_id.1,district,neighbourhood_id.1,neighbourhood
0,268917040,Tierarztpraxis am Urban,veterinary,Baerwaldstraße,69,10961.0,Berlin,None,None,None,...,None,52.495684,13.405233,2,27,"Baerwaldstraße 69, 10961.0 Berlin",2,Friedrichshain-Kreuzberg,27,Kreuzberg
1,299795048,Dr. med. vet. Elke Hartwig,veterinary,Straße 48,67,13125.0,Berlin,+49 30 9437820,http://www.tierarztpraxis-hartwig.de/,None,...,None,52.606286,13.479555,7,24,"Straße 48 67, 13125.0 Berlin",7,Pankow,24,Karow
2,347294456,Tierarztpraxis Dr. Bernhard Sörensen,veterinary,Königsberger Straße,36,12207.0,Berlin,+49 30 7738321,https://www.tierarztpraxis-soerensen.de/,None,...,None,52.429722,13.320133,10,32,"Königsberger Straße 36, 12207.0 Berlin",10,Steglitz-Zehlendorf,32,Lichterfelde
3,394867279,Tierarztpraxis Jeanette Koepsel,veterinary,None,None,NaN,Berlin,None,None,None,...,None,52.535199,13.270573,9,52,Berlin,9,Spandau,52,Siemensstadt
4,411550894,Kleintierarztpraxis Berlin Kaulsdorf,veterinary,Planitzstraße,19,12621.0,Berlin,+49 30 53018585,https://www.tierarzt-kaulsdorf.de/,info@tierarzt-kaulsdorf.de,...,None,52.509511,13.589635,4,25,"Planitzstraße 19, 12621.0 Berlin",4,Marzahn-Hellersdorf,25,Kaulsdorf
5,581887970,Eva Klein,veterinary,None,None,NaN,Berlin,None,None,None,...,None,52.428817,13.193529,10,43,Berlin,10,Steglitz-Zehlendorf,43,Nikolassee
6,603392630,Klein- und Heimtierklinik,veterinary,Oertzenweg,19b,14163.0,Berlin,None,https://www.vetmed.fu-berlin.de/einrichtungen/...,None,...,yes,52.430298,13.237360,10,67,"Oertzenweg 19b, 14163.0 Berlin",10,Steglitz-Zehlendorf,67,Zehlendorf
7,702363921,Tierarztpraxis im Frauenviertel - Firchow,veterinary,None,None,NaN,Berlin,None,None,None,...,None,52.404260,13.507166,6,49,Berlin,6,Neukölln,49,Rudow
8,703135335,Kleintierpraxis Kladow,veterinary,Zingerleweg,29,14089.0,Berlin,+49 30 3653374,http://www.kleintierpraxis-kladow.de,info@kleintierpraxis-kladow.de,...,None,52.460888,13.121554,9,26,"Zingerleweg 29, 14089.0 Berlin",9,Spandau,26,Kladow
9,978560965,Carsten Schiller,veterinary,Schnellerstraße,97,12439.0,Berlin,None,None,None,...,None,52.453108,13.524567,12,41,"Schnellerstraße 97, 12439.0 Berlin",12,Treptow-Köpenick,41,Niederschöneweide


In [18]:
pd.read_sql(text("SELECT COUNT(*) AS rows FROM berlin_source_data.veterinary_clinics_martin_svitek"), engine)



,rows
0,178
